<a href="https://colab.research.google.com/github/Tarun2733/game-recommendation-system/blob/main/gameModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Pandas and Numpy

In [1]:
import pandas as pd
import numpy as np

In [4]:
games = pd.read_csv('/content/steam.csv')
tags_df = pd.read_csv('/content/steamspy_tag_data.csv')
desc_df = pd.read_csv('/content/steam_description_data.csv')
media_df = pd.read_csv('/content/steam_media_data.csv')



In [5]:
games.head(1)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19


In [6]:
tags_df.head(1)

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550


In [7]:
desc_df.head(1)

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


In [8]:
desc_df = desc_df.rename(columns={'steam_appid': 'appid'})

In [9]:
desc_df.head(1)

,appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


In [10]:
media_df.head(1)

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN


In [11]:
games = games.merge(desc_df,on = 'appid')

In [12]:
games.shape

(27075, 21)

In [13]:
games.head(1)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,124534,3339,17612,317,10000000-20000000,7.19,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...


Select the columns that will be used:
appid,
name,
developer,
categories,
genre,
steamspy_tags,



In [14]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   appid                 27075 non-null  int64  
 1   name                  27075 non-null  object 
 2   release_date          27075 non-null  object 
 3   english               27075 non-null  int64  
 4   developer             27074 non-null  object 
 5   publisher             27061 non-null  object 
 6   platforms             27075 non-null  object 
 7   required_age          27075 non-null  int64  
 8   categories            27075 non-null  object 
 9   genres                27075 non-null  object 
 10  steamspy_tags         27075 non-null  object 
 11  achievements          27075 non-null  int64  
 12  positive_ratings      27075 non-null  int64  
 13  negative_ratings      27075 non-null  int64  
 14  average_playtime      27075 non-null  int64  
 15  median_playtime    

In [15]:
print(games.columns.tolist())


['appid', 'name', 'release_date', 'english', 'developer', 'publisher', 'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings', 'negative_ratings', 'average_playtime', 'median_playtime', 'owners', 'price', 'detailed_description', 'about_the_game', 'short_description']


In [16]:
games = games[['appid', 'name','developer','steamspy_tags']]

In [17]:
games.head()

,appid,name,developer,steamspy_tags
0,10,Counter-Strike,Valve,Action;FPS;Multiplayer
1,20,Team Fortress Classic,Valve,Action;FPS;Multiplayer
2,30,Day of Defeat,Valve,FPS;World War II;Multiplayer
3,40,Deathmatch Classic,Valve,Action;FPS;Multiplayer
4,50,Half-Life: Opposing Force,Gearbox Software,FPS;Action;Sci-fi


In [18]:
games.isnull().sum()

,0
appid,0
name,0
developer,1
steamspy_tags,0


In [19]:
games.dropna(inplace=True)

In [20]:
games.duplicated().sum()

np.int64(0)

In [21]:
# games['short_description'][0]

In [22]:
games['steamspy_tags'][0]

'Action;FPS;Multiplayer'

In [23]:
def string_to_list(st):
  if pd.isnull(st):
    return []
  return st.lower().replace(';',' ').split()


In [24]:
# games['short_description'] = games['short_description'].apply(string_to_list)

In [25]:
games.head(1)

,appid,name,developer,steamspy_tags
0,10,Counter-Strike,Valve,Action;FPS;Multiplayer


In [26]:
games['developer'] = games['developer'].apply(string_to_list)
games['steamspy_tags'] = games['steamspy_tags'].apply(string_to_list)
games['new_name'] = games['name'].apply(string_to_list)


In [27]:
games.head(1)

,appid,name,developer,steamspy_tags,new_name
0,10,Counter-Strike,[valve],"[action, fps, multiplayer]",[counter-strike]


In [28]:
games['tags'] = games['new_name']+games['developer']+games['steamspy_tags']

In [29]:
games.head(1)

,appid,name,developer,steamspy_tags,new_name,tags
0,10,Counter-Strike,[valve],"[action, fps, multiplayer]",[counter-strike],"[counter-strike, valve, action, fps, multiplayer]"


In [30]:
new_df = games

In [31]:
new_df

,appid,name,developer,steamspy_tags,new_name,tags
0,10,Counter-Strike,[valve],"[action, fps, multiplayer]",[counter-strike],"[counter-strike, valve, action, fps, multiplayer]"
1,20,Team Fortress Classic,[valve],"[action, fps, multiplayer]","[team, fortress, classic]","[team, fortress, classic, valve, action, fps, ..."
2,30,Day of Defeat,[valve],"[fps, world, war, ii, multiplayer]","[day, of, defeat]","[day, of, defeat, valve, fps, world, war, ii, ..."
3,40,Deathmatch Classic,[valve],"[action, fps, multiplayer]","[deathmatch, classic]","[deathmatch, classic, valve, action, fps, mult..."
4,50,Half-Life: Opposing Force,"[gearbox, software]","[fps, action, sci-fi]","[half-life:, opposing, force]","[half-life:, opposing, force, gearbox, softwar..."
...,...,...,...,...,...,...
27070,1065230,Room of Pandora,"[shen, jiawei]","[adventure, indie, casual]","[room, of, pandora]","[room, of, pandora, shen, jiawei, adventure, i..."
27071,1065570,Cyber Gun,"[semyon, maximov]","[action, indie, adventure]","[cyber, gun]","[cyber, gun, semyon, maximov, action, indie, a..."
27072,1065650,Super Star Blast,[entwicklerx],"[action, indie, casual]","[super, star, blast]","[super, star, blast, entwicklerx, action, indi..."
27073,1066700,New Yankee 7: Deer Hunters,"[yustas, game, studio]","[indie, casual, adventure]","[new, yankee, 7:, deer, hunters]","[new, yankee, 7:, deer, hunters, yustas, game,..."


In [32]:
df = games[['appid','name','tags']]

In [33]:
df.head()

,appid,name,tags
0,10,Counter-Strike,"[counter-strike, valve, action, fps, multiplayer]"
1,20,Team Fortress Classic,"[team, fortress, classic, valve, action, fps, ..."
2,30,Day of Defeat,"[day, of, defeat, valve, fps, world, war, ii, ..."
3,40,Deathmatch Classic,"[deathmatch, classic, valve, action, fps, mult..."
4,50,Half-Life: Opposing Force,"[half-life:, opposing, force, gearbox, softwar..."


In [34]:
#convert list to string
new_df["developer"] = new_df["developer"].apply(lambda x: " ".join(x))



In [35]:
new_df.head(1)

,appid,name,developer,steamspy_tags,new_name,tags
0,10,Counter-Strike,valve,"[action, fps, multiplayer]",[counter-strike],"[counter-strike, valve, action, fps, multiplayer]"


In [36]:
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer

label = LabelEncoder()
mlb = MultiLabelBinarizer()


In [37]:
#label encode the developer field
new_df["developer"] = label.fit_transform(new_df["developer"])

In [38]:
new_df.head(1)

,appid,name,developer,steamspy_tags,new_name,tags
0,10,Counter-Strike,15604,"[action, fps, multiplayer]",[counter-strike],"[counter-strike, valve, action, fps, multiplayer]"


In [39]:
new_df = new_df.join(pd.DataFrame(mlb.fit_transform(new_df['steamspy_tags']),
                          columns=mlb.classes_,
                          index=new_df.index))

In [40]:
new_df.head(1)

,appid,name,developer,steamspy_tags,new_name,tags,&,'em,1980s,1990's,...,web,werewolves,western,with,word,workshop,world,wrestling,your,zombies
0,10,Counter-Strike,15604,"[action, fps, multiplayer]",[counter-strike],"[counter-strike, valve, action, fps, multiplayer]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
print(new_df.columns.tolist())

['appid', 'name', 'developer', 'steamspy_tags', 'new_name', 'tags', '&', "'em", '1980s', "1990's", '2.5d', '2d', '3', '360', '3d', '4', '40k', '4x', '6dof', 'abstract', 'access', 'action', 'action-adventure', 'adventure', 'agriculture', 'aliens', 'alternate', 'america', 'and', 'animation', 'anime', 'arcade', 'arena', 'arts', 'assassin', 'atmospheric', 'attack', 'audio', 'awkward', 'base-building', 'baseball', 'basketball', 'batman', 'battle', 'beat', 'beautiful', 'benchmark', 'bikes', 'blood', 'bmx', 'board', 'book', 'bowling', 'builder', 'building', 'bullet', 'capitalism', 'card', 'cartoon', 'cartoony', 'casual', 'cats', 'character', 'chess', 'choices', 'choose', 'cinematic', 'city', 'class-based', 'classic', 'click', 'clicker', 'co-op', 'cold', 'colorful', 'combat', 'comedy', 'comic', 'competitive', 'content', 'control', 'controller', 'controls', 'conversation', 'crafting', 'crawler', 'crime', 'croft', 'crpg', 'cult', 'customization', 'cute', 'cyberpunk', 'cycling', 'dark', 'dating',

In [42]:
cols_to_drop = ['steamspy_tags', 'new_name', 'tags','&',"'em"]
new_df.drop(columns=[i for i in cols_to_drop if i in new_df.columns], inplace=True)


In [43]:
new_df.head(1)

,appid,name,developer,1980s,1990's,2.5d,2d,3,360,3d,...,web,werewolves,western,with,word,workshop,world,wrestling,your,zombies
0,10,Counter-Strike,15604,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
remaining = [i for i in mlb.classes_ if i in new_df.columns]
features = ['developer']+ remaining
v = new_df[features].values

In [45]:
new_df.shape

(27074, 387)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
sim_mat= cosine_similarity(v,dense_output=False)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
import re

ps = PorterStemmer()
#stemming and cleaning
def stem(t):
  t = re.sub('[^a-zA-Z]', ' ', t)
  t = t.lower()
  t = t.split()
  lst = []
  for word in t:
    lst.append(ps.stem(word))

  return " ".join(lst)

In [ ]:
df['tags'][0]

['counter-strike', 'valve', 'action', 'fps', 'multiplayer']

In [ ]:
df['tags'][1]

['team', 'fortress', 'classic', 'valve', 'action', 'fps', 'multiplayer']

In [ ]:
ps.stem('loved')

'love'

In [ ]:
# convert list to string
df['tags'] = df['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

<ipython-input-34-68cc1262373c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)


In [ ]:
df.head()

,appid,name,tags
0,10,Counter-Strike,counter-strike valve action fps multiplayer
1,20,Team Fortress Classic,team fortress classic valve action fps multipl...
2,30,Day of Defeat,day of defeat valve fps world war ii multiplayer
3,40,Deathmatch Classic,deathmatch classic valve action fps multiplayer
4,50,Half-Life: Opposing Force,half-life: opposing force gearbox software fps...


In [ ]:
df['tags'] = df['tags'].apply(stem)

<ipython-input-36-9c043b76a2e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [ ]:
df.shape


(27074, 3)

In [ ]:
#stemmed tags
df['tags'][0]

'counter strike valv action fp multiplay'

Vectorization using TF-IDF


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')

In [ ]:
matrix = tfidf.fit_transform(df['tags']).toarray()

In [ ]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
tfidf.get_feature_names_out()

array(['aaron', 'aartform', 'ab', ..., 'zov', 'zovgam', 'zup'],
      dtype=object)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_matrix = cosine_similarity(matrix)

In [ ]:
similarity_matrix.shape

(27074, 27074)

In [ ]:
#recommend function
def recommend(game):
  game = game.lower()
  idx = df[df['name'].str.lower() == game].index[0]

  distances = similarity_matrix[idx]
  lst = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  # for i in lst:
  #   print(df.iloc[i[0]]['name'])
  recoms = [df.iloc[i[0]]['name']for i in lst]
  return recoms

In [ ]:
recommend('Team Fortress Classic')

['Super Fancy Pants Adventure',
 'Fat City',
 'Need for Synthol',
 'Atom Universe',
 'ERISLE']

In [ ]:
df.head()

,appid,name,tags
0,10,Counter-Strike,counter strike valv action fp multiplay
1,20,Team Fortress Classic,team fortress classic valv action fp multiplay
2,30,Day of Defeat,day of defeat valv fp world war ii multiplay
3,40,Deathmatch Classic,deathmatch classic valv action fp multiplay
4,50,Half-Life: Opposing Force,half life oppos forc gearbox softwar fp action...


In [ ]:
# print(recommend("Half-Life 2"))


In [ ]:
print(df[df['name'] == "Half-Life 2"]['tags'].values[0])

half life valv fp action sci fi


In [ ]:
for i, title in enumerate(recommend("Half-Life 2"), 1):
    print(f"{i}. {title}")


1. Trends
2. My Golf
3. Warlocks vs Shadows
4. Puppies vs Undead
5. GoNNER
